# CS 3120 Machine Learning HW1

Devon DeJohn, Spring 2020

In [1]:
import sys
sys.path.append('../')
from source import k_nearest_neighbors
model = k_nearest_neighbors.Model("../data/animals")

SyntaxError: invalid syntax (k_nearest_neighbors.py, line 201)

## Imports

```python
import cv2
import pathlib
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report as crep
from random import shuffle
```

## Train, Test, Validate

My default model's parameters are slightly different than `sklearn`'s model, as noted in the initial call to `super().__init__()`:

```python
DEFAULT = {
    "n_neighbors": 3,
    "metric": "manhattan",
    "weights": "distance",
    "n_jobs": -1
}

class Model(KNeighborsClassifier):
    """A container for a KNN classifier"""
    def __init__(self, path: str):
        super().__init__(**DEFAULT)
        self.dims = (16,16)
        self.labels = {}
        self.path = path
        self.parts = ["train", "test", "validate"]
        self.load_data()
        self.fit(self.train.X, self.train.Y)
    # end
```

I also decided to implement my own version of `train_test_split` that supports multiple partitions, not just two:

In [ ]:
model.datasets()

## K=3, using the $\ell_1$-norm

For `n_neighbors=3`, using the `manhattan` distance metric, or the $\ell_1$-norm, we have:

In [ ]:
model.report()

## Retraining

I added the ability to retrain the model directly through the `Model` type by making `sklearn.KNeighborsClassifier` a super class of `Model`, so that the classifier's attributes are accessible via the `Model` type.

## Performance

Taking the cartesian product of these parameters, we can retrain our model on every combination, and measure the mean accuracy:

```python
dims = [8, 16, 32, 64]
neighbors = [3, 5, 7, 9]
metrics = ["manhattan", "euclidean"]
weights = ["uniform", "distance"]
```

Each time `Model.retrain()` is called, the seed for the `random` module is reset, so the testing data are the same across each run.

Each subplot below represents a metric (columnar), and a weighting algorithm (tabular). The horizontal axis of each subplot represents the pixel dimensions of the rescaled image data, and the vertical axis represents the number of neighbors counted in voting.

In [ ]:
model.cycle_params()